In [1]:
import openferro as of
from openferro.interaction import *
from openferro.simulation import *
from openferro.engine import *
from openferro.ewald import dipole_dipole_ewald
from matplotlib import pyplot as plt
import json

In [5]:
L =12
config = json.load(open('../model_configs/BaTiO3.json'))
latt_vecs = jnp.eye(3) * config['lattice']['a1']
latt = of.BravaisLattice3D(L, L, L, latt_vecs[0], latt_vecs[1], latt_vecs[2])
bto = of.System(latt, pbc=True)
## define fields
dipole_field = bto.add_field(name="dipole", ftype="Rn", dim=3, value=0.1, mass = 10)
lstrain_field = bto.add_field(name="lstrain", ftype="local_strain", value=0.0, mass = 10)
gstrain_field = bto.add_field(name="gstrain", ftype="global_strain", value=jnp.zeros(6), mass = 1000)

## define Hamiltonian
bto.add_self_interaction('self_onsite', field_name="dipole", energy_engine=self_energy_onsite_isotropic, parameters=config["onsite"], enable_jit=True)
bto.add_self_interaction('short_range_1', field_name="dipole", energy_engine=short_range_1stnn_isotropic, parameters=config["short_range"], enable_jit=True)
bto.add_self_interaction('short_range_2', field_name="dipole", energy_engine=short_range_2ednn_isotropic, parameters=config["short_range"], enable_jit=True)
bto.add_self_interaction('short_range_3', field_name="dipole", energy_engine=short_range_3rdnn_isotropic, parameters=config["short_range"], enable_jit=True)
bto.add_self_interaction('dipole_dipole', field_name="dipole", energy_engine=dipole_dipole_ewald, parameters=config["born"]|config["lattice"], enable_jit=True)

homo_config = config["elastic"].copy()
homo_config['N'] = L**3
# bto.add_self_interaction('homo_elastic', field_name="gstrain", energy_engine=homo_elastic_energy, parameters=homo_config, enable_jit=True)
# bto.add_mutual_interaction('homo_strain_dipole', field_name1="gstrain", field_name2="dipole", energy_engine=homo_strain_dipole_interaction, parameters=config["elastic_dipole"], enable_jit=True)

# bto.add_mutual_interaction('elastic', field_name1="lstrain", field_name2="gstrain", energy_engine=elastic_energy, parameters=config["elastic"], enable_jit=True)
# bto.add_mutual_interaction('inhomo_strain_dipole', field_name1="lstrain", field_name2="dipole", energy_engine=inhomo_strain_dipole_interaction, parameters=config["elastic_dipole"], enable_jit=True)

interaction_name = []
for interaction in bto._self_interaction_dict:
    interaction_name.append(interaction)
for interaction in bto._mutual_interaction_dict:
    interaction_name.append(interaction)


In [6]:

print('mass:', bto.get_field_by_name('dipole').get_mass().flatten())
bto.update_force()
print('max force before optimization',jnp.abs(bto.get_field_by_name("dipole").get_force()).max())
print('average field before optimization',bto.get_field_by_name("dipole").get_values().mean())
for interaction in interaction_name:
    print('E({})={}eV'.format( interaction, bto.calc_energy_by_name(interaction)))
minimizer = MDMinimize(bto, max_iter=200, tol=1e-5, dt=0.001)
minimizer.minimize()

equilibrium_field = bto.get_field_by_name("dipole").get_values().copy()
print('max force after optimization',jnp.abs(bto.get_field_by_name("dipole").get_force()).max())
print('average field after optimization',bto.get_field_by_name("dipole").get_values().mean())
for interaction in interaction_name:
    print('E({})={}eV'.format( interaction, bto.calc_energy_by_name(interaction)))
print('field:', equilibrium_field[:3,:3,:3])

mass: [10. 10. 10. ... 10. 10. 10.]
max force before optimization 46.108517
average field before optimization 0.10000395
E(self_onsite)=372.35321044921875eV
E(short_range_1)=-72.6681900024414eV
E(short_range_2)=104.4072036743164eV
E(short_range_3)=74.23656463623047eV
E(dipole_dipole)=-12516.1455078125eV
max force after optimization 0.0002746582
average field after optimization 0.8340422
E(self_onsite)=441660.46875eV
E(short_range_1)=-5058.396484375eV
E(short_range_2)=7264.14306640625eV
E(short_range_3)=5164.69384765625eV
E(dipole_dipole)=-870853.5625eV
field: [[[[0.8341043 0.8341043 0.8341043]
   [0.8341043 0.8341043 0.8341043]
   [0.8341043 0.8341043 0.8341043]]

  [[0.8341043 0.8341043 0.8341043]
   [0.8341043 0.8341043 0.8341043]
   [0.8341043 0.8341043 0.8341043]]

  [[0.8341043 0.8341043 0.8341043]
   [0.8341043 0.8341043 0.8341043]
   [0.8341043 0.8341043 0.8341043]]]


 [[[0.8341043 0.8341043 0.8341043]
   [0.8341043 0.8341043 0.8341043]
   [0.8341043 0.8341043 0.8341043]]

  [[

In [7]:
temp_list = [ 20,40,60,80,100,120,140]
# average_field = np.zeros(len(temp_list))
log_freq = 100
total_time = 100000
dt = 0.1
relax_steps = int(1000/dt)
total_steps = int(total_time / dt)
niters = total_steps // log_freq
field_history = []
for it, temperature in enumerate(temp_list):
    simulation = SimulationNVTLangevin(pto, dt=dt, temperature=temperature, gamma=1/(100*dt))
    simulation.system.get_field_by_name('dipole').set_values(equilibrium_field.copy())
    simulation.init_velocity(mode='gaussian')
    simulation.step(relax_steps)
    print('T={}K'.format(temperature))
    average_field = []
    for ii in range(niters):
        simulation.step(log_freq)
        # print('velocity:', pto.get_field_by_name('dipole').get_velocity().flatten())
        # print('field:', pto.get_field_by_name('dipole').get_values().flatten())
        pot_E = pto.calc_potential_energy()
        kin_E = pto.calc_kinetic_energy()
        total_E = pot_E + kin_E
        # average_field[it] += pto.get_field_by_name('dipole').get_values().mean() / niters
        average_field.append(pto.get_field_by_name('dipole').get_values().mean())
        # print('=================T={}K, iter={}======================='.format(temperature, ii))
        # print('pot energy:', pot_E)
        # print('kine energy:', kin_E)
        # print('total energy:', total_E)
        # print('temperature:', pto.calc_temp_by_name('dipole'))
        # print('average field:', average_field)
        # print('field:', pto.get_field_by_name('dipole').get_values().flatten())
    field_history.append(average_field)
field_history = jnp.array(field_history)


T=300K
T=400K
T=500K
T=600K
T=700K
T=800K
T=900K
